# Vanessa Williams
# Week 11

# Section 1  
-Import sqlite3 and create a cursor  
-Create the sensors table (pg 180)  
-Write a query to confirm your table was created – print the results  
-Insert random data into the table you created and then select all the results from the table (Select *)  
-Drop the table and confirm it has been dropped based on your previous query  
-Close your connection  

In [1]:
import sqlite3

#creating sensors table with five columns
with sqlite3.connect(":memory:") as con:
    c = con.cursor()
    c.execute('''CREATE TABLE sensors (date text, city text, code text, sensor_id real, temperature real)''')
#listing the sql table
    for table in c.execute("SELECT name FROM sqlite_master WHERE type = 'table'"):
        print("Table", table[0])
#inserting random data
    c.execute("INSERT INTO sensors VALUES ('2024-07-11','Utrecht','Green',42,15.14)")
    c.execute("SELECT * FROM sensors")
    print(c.fetchone())
#dropping the table
    con.execute("DROP TABLE sensors")

    print("# of tables", c.execute("SELECT COUNT(*) FROM sqlite_master WHERE type = 'table'").fetchone()[0])

    c.close()

Table sensors
('2024-07-11', 'Utrecht', 'Green', 42.0, 15.14)
# of tables 0


# Section 2
-Create a list of tuples to form the pandas DataFrame  
-Create a table without specifying data types  
-Insert records from a list of tuples – insert all the rows into the table and show the row count  
-Delete the records where the sunactivity is greater than 20  
-Write a query to select all data where the year is less than 1732  

In [2]:
import statsmodels.api as sm
from pandas.io.sql import read_sql
import sqlite3

#create an in-memory SQLite database connection
with sqlite3.connect(":memory:") as con:
    c = con.cursor()
#load sunspot data using the statsmodels library
    data_loader = sm.datasets.sunspots.load_pandas()
    df = data_loader.data
#convert to list of tuples
    rows = [tuple(x) for x in df.values]
#Create table with two columns
    con.execute("CREATE TABLE sunspots(year, sunactivity)")
#insert the data
    con.executemany("INSERT INTO sunspots(year, sunactivity) VALUES (?, ?)", rows)
    c.execute("SELECT COUNT(*) FROM sunspots")
    print(c.fetchone())
#drop sunactivity greater than 20
    print("Deleted", con.execute("DELETE FROM sunspots where sunactivity > 20").rowcount, "rows")
#print data for years before 1732
    print(read_sql("SELECT * FROM sunspots where year < 1732", con))   
#drop the table and close the cursor
    con.execute("DROP TABLE sunspots")
    c.close()

(309,)
Deleted 217 rows
      year  sunactivity
0   1700.0          5.0
1   1701.0         11.0
2   1702.0         16.0
3   1707.0         20.0
4   1708.0         10.0
5   1709.0          8.0
6   1710.0          3.0
7   1711.0          0.0
8   1712.0          0.0
9   1713.0          2.0
10  1714.0         11.0
11  1723.0         11.0


# Section 3
-Install sqlalchemy  
-Define the superclass  
-Create the DBSession  
-Create 2 stations and commit the session  
-Insert a sensor record  

In [8]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sqlalchemy import Column, ForeignKey, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy import UniqueConstraint

#define superclass
Base = declarative_base()
#create station class for station table
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    name = Column(String(14), nullable=False, unique=True)

    def __repr__(self):
        return "Id=%d name=%s" %(self.id, self.name)
    
#create sensor class for sensor table
class Sensor(Base):
    __tablename__ = 'sensor'
    id = Column(Integer, primary_key=True)
    last = Column(Integer)
    multiplier = Column(Float)
    station_id = Column(Integer, ForeignKey('station.id'))
    station = relationship(Station)

    def __repr__(self):
        return "Id=%d last=%d multiplier=%.1f station_id=%d" 
# %(self.id, self.last, self.multiplier, self.station_id)

#define main block to represent this script to be used with other code
if __name__ == "__main__":
    print("This script is used by code further down in this notebook.")

This script is used by code further down in this notebook.


# Section 4
-Create an engine from a URI  
-Create the station and sensor tables with the engine we just created  
-Create a DBSession object  
-Select the first row in the station table  
-Select all the stations  
-Select all the sensors  
-Select the first sensor, which belongs to the first station  
-Query the station table with pandas read_sql()  

In [6]:
from sqlalchemy.orm import sessionmaker

#creat dbsesssion object
def populate(engine):
    Base.metadata.bind = engine
    DBSession = sessionmaker(bind=engine)
    session = DBSession()

    # Check if 'De Bilt' station exists, if not, add it
    de_bilt = session.query(Station).filter_by(name='De Bilt').first()
    if not de_bilt:
        de_bilt = Station(name='De Bilt')
        session.add(de_bilt)
        print("Added Station", de_bilt)

    # Check if 'Utrecht' station exists, if not, add it
    utrecht = session.query(Station).filter_by(name='Utrecht').first()
    if not utrecht:
        utrecht = Station(name='Utrecht')
        session.add(utrecht)
        print("Added Station", utrecht)

    session.commit()

    # Check if sensor exists, if not, add it
    sensor = session.query(Sensor).filter_by(station=de_bilt).first()
    if not sensor:
        temp_sensor = Sensor(last=20, multiplier=0.1, station=de_bilt)
        session.add(temp_sensor)
        session.commit()
        print("Added Sensor", temp_sensor)
#define main block to represent this script to be used with other code
if __name__ == "__main__":
    print("This script is used by code further down in this notebook")

This script is used by code further down in this notebook


In [8]:
import os
import pandas as pd
from pandas.io.sql import read_sql

#create engine
engine = create_engine('sqlite:///demo.db')
#create all tables defined in base metadata
Base.metadata.create_all(engine)
#call function to add data to the dataset
populate(engine)
#bind the engine to the base metadata
Base.metadata.bind = engine
#create session factory and bind it to the engine
DBSession = sessionmaker()
DBSession.bind = engine
#create the session object
session = DBSession()

station = session.query(Station).first()
#Query the data
print("Query 1", session.query(Station).all())
print("Query 2", session.query(Sensor).all())
print("Query 3", session.query(Sensor).filter(Sensor.station == station).one())

# Use the engine directly with pandas read_sql
print(pd.read_sql("SELECT * FROM station", engine))

try:
    os.remove('demo.db')
    print("Deleted demo.db")
except OSError:
    pass

Query 1 [Id=1 name=De Bilt, Id=2 name=Utrecht]
Query 2 [Id=%d last=%d multiplier=%.1f station_id=%d]
Query 3 Id=%d last=%d multiplier=%.1f station_id=%d
   id     name
0   1  De Bilt
1   2  Utrecht


Code Attribution:  
Title: 11.2 Exercise  
Author: Armando Fandango  
Date: 07/11/2024  
Modified By: Vanessa Williams  
Description: Databases